In [88]:
import pandas as pd
import librosa
import numpy as np

In [7]:
df = pd.read_csv('../../SQLqueries/fine_path_species_label.csv', header = None)

In [8]:
df.columns = ['label_id', 'audio_id', 'fine_start_time', 'fine_end_time', 'species', 'sound_type', 'path']

In [12]:
# Load appropriate audio from path
example_audio = df.path[0]

In [97]:
x = []

In [107]:
for i, item in enumerate(range(len(df))):
    x.append(librosa.load('../../Data' + df["path"][i].strip().strip("\'"),
                offset=df["fine_start_time"][i],
                duration=df["fine_end_time"][i] - df["fine_start_time"][i]))

ValueError: Input signal length=0 is too small to resample from 44100->22050

In [105]:
x

[(array([1.0143545e-02, 7.1876207e-03, 6.0061691e-05, ..., 8.6885719e-03,
         5.3000557e-03, 0.0000000e+00], dtype=float32),
  22050),
 (array([-0.00544651, -0.00630925, -0.00636242, ...,  0.00205116,
          0.00132131,  0.        ], dtype=float32),
  22050),
 (array([-0.00247971, -0.00247937, -0.00191444, ..., -0.0013788 ,
         -0.00081036,  0.        ], dtype=float32),
  22050),
 (array([ 0.00165599,  0.00179809,  0.00158497, ..., -0.00289071,
         -0.00201089,  0.        ], dtype=float32),
  22050),
 (array([-4.7252593e-03, -6.6203903e-03, -8.0663767e-03, ...,
         -1.7447592e-04, -1.4995014e-06,  0.0000000e+00], dtype=float32),
  22050),
 (array([-0.0279984 , -0.031152  , -0.02863306, ..., -0.00197428,
         -0.00115601,  0.        ], dtype=float32),
  22050),
 (array([-0.01067041, -0.01257473, -0.01240105, ..., -0.01736164,
         -0.01203549,  0.        ], dtype=float32),
  22050),
 (array([-0.01296087, -0.01543915, -0.01546721, ...,  0.00813158,
        

In [113]:
df["fine_start_time"][10]

90.825463

In [112]:
df["fine_end_time"][10]

91.150249

In [109]:
i

1560

In [108]:
df["fine_end_time"][i] - df["fine_start_time"][i]

0.0